# Code:Nation - Desafio: Melhores colocados ENEM

O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o [Dicionário dos Microdados do Enem 2016](https://s3-us-west-1.amazonaws.com/acceleration-assets-highway/data-science/dicionario-de-dados.zip).


Muitas universidades brasileiras utilizam o ENEM para selecionar seus futuros alunos e alunas. Isto é feito com uma média ponderada das notas das provas de matemática, ciências da natureza, linguagens e códigos, ciências humanas e redação. Determine os 20 melhores colocados, por ordem, para os pesos abaixo:

- matemática: 3
- ciências da natureza: 2
- linguagens e códigos: 1.5
- ciências humanas: 1
- redação: 3

## 1. O código

Nesta seção, será implementada a solução para o desafio proposto.

### 1.1. Importando as dependências

Abaixo, importo a biblioteca Pandas, a qual ajudará a solucionar o problema e, ainda, importo um módulo de constantes que criei chamado _peso_notas_, o qual tem o peso das notas definidas conforme mencionado acima.

In [9]:
import pandas as pd
import peso_notas as peso

### 1.2. Importando o conjunto de dados e definindo as colunas de interesse

Esse conjunto de dados é muito grande e apenas algumas colunas nos interessam no momento, sendo elas:
- a coluna do número de inscrição do aluno
- a coluna da nota da prova de matemática
- a coluna da nota da prova de ciências da natureza
- a coluna da nota da prova de ciências humanas
- a coluna da nota da prova de linguagens e códigos
- e a coluna da nota da redação
Então, crio um vetor com os nomes das colunas de interesse (após analisar o [Dicionário dos Microdados do Enem 2016](https://s3-us-west-1.amazonaws.com/acceleration-assets-highway/data-science/dicionario-de-dados.zip), pude identificar o nome dessas colunas. E em seguida, carrego as informações do arquivo `train.csv`, passando o parâmetro `usecols` com as colunas que desejo utilizar.

Para garantir que tudo foi carregado corretamente, exibo os primeiros cinco registros do conjunto de dados.

In [2]:
cols = ['NU_INSCRICAO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']
df = pd.read_csv('train.csv', usecols=cols)
df.head()

,NU_INSCRICAO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
0,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,436.3,495.4,581.2,399.4,520.0
1,2c3acac4b33ec2b195d77e7c04a2d75727fad723,474.5,544.1,599.0,459.8,580.0
2,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,NaN,NaN,NaN,NaN,NaN
3,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,NaN,NaN,NaN,NaN,NaN
4,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,NaN,NaN,NaN,NaN,NaN


Em seguida, exibo as informações do conjunto de dados. Pode ser visto que possuem 13730 registros/linhas, porém, nem todas as colunas estão preenchidas (algumas com 10341 e outras com 10133).

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13730 entries, 0 to 13729
Data columns (total 6 columns):
NU_INSCRICAO       13730 non-null object
NU_NOTA_CN         10341 non-null float64
NU_NOTA_CH         10341 non-null float64
NU_NOTA_LC         10133 non-null float64
NU_NOTA_MT         10133 non-null float64
NU_NOTA_REDACAO    10133 non-null float64
dtypes: float64(5), object(1)
memory usage: 643.7+ KB


### 1.3. Limpando os dados

Como pode ser notado nos resultadso acima, existem valores que não são válidos para a análise (NaN). Neste caso eles são valores nulos, os quais poderão comprometer todo o esforço, caso não sejam removidos. 
Então, abaixo removo todos os valores nulos do conjunto de dados e em seguida, exibo novamente as informações, apenas para garantir que foi devidamente aplicada a técnica.

In [4]:
df.dropna(thresh=6, inplace=True)
df.head()

,NU_INSCRICAO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
0,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,436.3,495.4,581.2,399.4,520.0
1,2c3acac4b33ec2b195d77e7c04a2d75727fad723,474.5,544.1,599.0,459.8,580.0
5,a37c99ec251d4f6e8ddbeabadf1c87fdbfddc4d1,439.7,583.2,410.9,364.5,620.0
6,63b4e128e9ffe8ab27f5d093db1976ef4f353e0a,420.1,604.2,484.5,529.2,560.0
7,2eb189d4912f64b19d1967e8e84b6141aba18770,619.6,625.8,611.2,566.7,620.0


Novamente, aqui exibo as informações do conjunto de dados. E agora pode ser visto que, considerando apenas os registros que possuam todas as informações, temos 10097 linhas no total!

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10097 entries, 0 to 13729
Data columns (total 6 columns):
NU_INSCRICAO       10097 non-null object
NU_NOTA_CN         10097 non-null float64
NU_NOTA_CH         10097 non-null float64
NU_NOTA_LC         10097 non-null float64
NU_NOTA_MT         10097 non-null float64
NU_NOTA_REDACAO    10097 non-null float64
dtypes: float64(5), object(1)
memory usage: 552.2+ KB


### 1.4. Calculando a nota final

Com o conjunto de dados limpo, agora é possível trabalhar nele para atingir o objetivo final: **determinar os 20 melhores colocados do ENEM 2016**. 

Então, para obtermos a média ponderada (ou, neste caso, a **nota final**), calculamos cada valor do conjunto de dados (as notas) pelo seu peso. Depois, soma todos os valores obtidos e divide-se pela soma dos pesos.

Por exemplo, pegamos a nota da prova de ciências naturais e **multiplicamos** pelo seu peso (que é 2), e fazemos isso para as demais notas e seus respectivos pesos (ciências humanas igual a 2, linguagens e códigos igual a 1.5, matemática igual a 3 e redação também igual a 3). Então, **somamos** os valores das notas e **dividimos** pela **soma** dos pesos (_2 + 1 + 1.5 + 3 + 3 = **10.5**_).

In [6]:
soma_dos_pesos = peso.NOTA_CN + peso.NOTA_CH + peso.NOTA_LC + peso.NOTA_MT + peso.NOTA_REDACAO
df['NU_NOTA_CN'] = df['NU_NOTA_CN'].apply(lambda nota_cn: nota_cn*peso.NOTA_CN)
df['NU_NOTA_CH'] = df['NU_NOTA_CH'].apply(lambda nota_ch: nota_ch*peso.NOTA_CH)
df['NU_NOTA_LC'] = df['NU_NOTA_LC'].apply(lambda nota_lc: nota_lc*peso.NOTA_LC)
df['NU_NOTA_MT'] = df['NU_NOTA_MT'].apply(lambda nota_mt: nota_mt*peso.NOTA_MT)
df['NU_NOTA_REDACAO'] = df['NU_NOTA_REDACAO'].apply(lambda nota_redacao: nota_redacao*peso.NOTA_REDACAO)
df['NOTA_FINAL'] = (df['NU_NOTA_CN'] + df['NU_NOTA_CH'] + df['NU_NOTA_LC'] + df['NU_NOTA_MT'] + df['NU_NOTA_REDACAO'])/soma_dos_pesos

### 1.5. Preparando para o envio

Conforme é solicitado na página do desafio, a resposta deve estar em um arquivo chamado _answer.csv_ com duas colunas: `NU_INSCRICAO` e `NOTA_FINAL`.

Abaixo, **ordeno** os dados pela coluna `NOTA_FINAL` de forma **decrescente**, pego do índice 0 ao 20 e seleciono apenas as colunas solicitadas.

In [7]:
df_answer = df.sort_values(by='NOTA_FINAL', ascending=False)[:20][['NU_INSCRICAO', 'NOTA_FINAL']]
df_answer

,NU_INSCRICAO,NOTA_FINAL
10801,848daf808904864c1ee5d51545a2539e2dcbe901,834.147619
3226,97166da67df0908861a868cf318b277c4bdbb0da,828.214286
3682,b733ccac4e5b7c1da0448f1dad7990ec0793b1d2,819.514286
9011,78ca18b6fea7ccd3e067b3945cf200f8686209a7,817.333333
4755,8588b01c2ff81ad3c9ff31cd89fc094896c4aa69,813.671429
12707,19982f867b8f23a829a2f738374770708ec7f263,809.938095
6039,17ae668ce9041105883e9db101d614f58d526f66,808.847619
10219,3326c76719930f9838bc9ef803fe1b73a1668723,804.304762
1509,41489fd85a5d0cb6e397fb5ebcd19d32e9265cb7,802.833333
3535,32767c1a152919a1f72fa91afe19bb7c559f2824,800.752381


E após montar o conjunto de dados de acordo com o que é solicitado, ele é **exportado para um arquivo** chamado _answer.csv_ **sem indexação**.

In [8]:
df_answer.to_csv('answer.csv', index=False)